运动规划算法有很多种，包括

![](./tg002.png)

组合法：将自由空间分成小块，然后将这些原子元素链接起来，解决运动规划问题；这种方法与我们直觉相符，去发现最初的近似解；但当环境规模扩大后，这种方法表现不佳；

位势场算法：是反馈算法，每个障碍物会产生一种反重力场，这使车辆不会去靠近它们。比如算法轨迹会绕过行人和自行车；
位势场算法的主要问题是，有时候它会把我们带入局部极小值中。难以找到最合适解；

最优控制法：通过算法来生成控制PoD，使用动态模型，为车辆，最初配置和最终配置建模，概算法需要我们生成一系列的输入，比如转向和减速，这些输入会把我们从开始配置带到最终配置。在此过程中，算法会优化与控制输入，车辆配置等相关的成本函数。比如最小汽油消耗。这种算法有多种实现方式，大部分都基于数值优化法；

但是要把所有与其他车辆相关的限制都很好地纳入考量，还要使算法很快地的得出结果，是相当困难的。

抽样算法：使用一个基于碰撞检测的模块，该模块探测自由空间，测试是否一个配置在其中会发生碰撞，不同于组合法和最优控制法(这两种方法均需要分析整个环境)，抽样法不需要检测所有自由空间，便可以发现一条路径，搜索过的路径保存在一个图结构中，然后使用图搜索算法例如$D^\star 或者A^\star$来搜索。

有两种方法可以认定是基于抽样的，

一是离散法：它依赖于构造良好的辅助集合或者输入，比如覆盖在我们配置空间上的一张格子图；

2概率统计法：它依赖连续配置空间的概率样本，待探索的可能配置或者状态集，可能是无穷多的，这让这类算法具有一种不错的性质，那就是它们是概率完整的，并且有时候概率的优化，意味着它们最终会找出一个解来，前提是给予足够的计算时间。

![](./tg005.png)

# 混合$A^\star$

和标准$A^\star$相比，混合$A^\star$牺牲了少许完整性和优化性，但是确保了可驾驶性。

![](./tg100.png)

而在实际应用中，混合$A^\star$效率极高，而且几乎每次都可以找到不错的路径。

Hybrid A* Heuristics

The paper Junior: The Stanford Entry in the Urban Challenge is a good read overall, but Section 6.3 - Free Form Navigation is especially good and goes into detail about how the Stanford team thought about heuristics for their Hybrid A* algorithm (which they tended to use in parking lots).

https://d17h27t6h515a5.cloudfront.net/topher/2017/July/595fe838_junior-the-stanford-entry-in-the-urban-challenge/junior-the-stanford-entry-in-the-urban-challenge.pdf


伪代码实现：
```
def expand(state, goal):
    next_states = []
    for delta in range(-35, 40, 5): 
        # Create a trajectory with delta as the steering angle using 
        # the bicycle model:

        # ---Begin bicycle model---
        delta_rad = deg_to_rad(delta)
        omega = SPEED/LENGTH * tan(delta_rad)
        next_x = state.x + SPEED * cos(theta)
        next_y = state.y + SPEED * sin(theta)
        next_theta = normalize(state.theta + omega)
        # ---End bicycle model-----

        next_g = state.g + 1
        next_f = next_g + heuristic(next_x, next_y, goal)

        # Create a new State object with all of the "next" values.
        state = State(next_x, next_y, next_theta, next_g, next_f)
        next_states.append(state)

    return next_states

def search(grid, start, goal):
    # The opened array keeps track of the stack of States objects we are 
    # searching through.
    opened = []
    # 3D array of zeros with dimensions:
    # (NUM_THETA_CELLS, grid x size, grid y size).
    closed = [[[0 for x in range(grid[0])] for y in range(len(grid))] 
        for cell in range(NUM_THETA_CELLS)]
    # 3D array with same dimensions. Will be filled with State() objects 
    # to keep track of the path through the grid. 
    came_from = [[[0 for x in range(grid[0])] for y in range(len(grid))] 
        for cell in range(NUM_THETA_CELLS)]

    # Create new state object to start the search with.
    x = start.x
    y = start.y
    theta = start.theta
    g = 0
    f = heuristic(start.x, start.y, goal)
    state = State(x, y, theta, 0, f)
    opened.append(state)

    # The range from 0 to 2pi has been discretized into NUM_THETA_CELLS cells. 
    # Here, theta_to_stack_number returns the cell that theta belongs to. 
    # Smaller thetas (close to 0 when normalized  into the range from 0 to 
    # 2pi) have lower stack numbers, and larger thetas (close to 2pi when 
    # normalized) have larger stack numbers.
    stack_num = theta_to_stack_number(state.theta)
    closed[stack_num][index(state.x)][index(state.y)] = 1

    # Store our starting state. For other states, we will store the previous 
    # state in the path, but the starting state has no previous.
    came_from[stack_num][index(state.x)][index(state.y)] = state

    # While there are still states to explore:
    while opened:
        # Sort the states by f-value and start search using the state with the 
        # lowest f-value. This is crucial to the A* algorithm; the f-value 
        # improves search efficiency by indicating where to look first.
        opened.sort(key=lambda state:state.f)
        current = opened.pop(0)

        # Check if the x and y coordinates are in the same grid cell 
        # as the goal. (Note: The idx function returns the grid index for 
        # a given coordinate.)
        if (idx(current.x) == goal[0]) and (idx(current.y) == goal.y):
            # If so, the trajectory has reached the goal.
            return path

        # Otherwise, expand the current state to get a list of possible 
        # next states.
        next_states = expand(current, goal)
        for next_s in next_states:
            # If we have expanded outside the grid, skip this next_s.
            if next_s is not in the grid:
                continue
            # Otherwise, check that we haven't already visited this cell and
            # that there is not an obstacle in the grid there.
            stack_num = theta_to_stack_number(next_s.theta)
            if closed[stack_num][idx(next_s.x)][idx(next_s.y)] == 0 
                and grid[idx(next_s.x)][idx(next_s.y)] == 0:
                # The state can be added to the opened stack.
                opened.append(next_s)
                # The stack_number, idx(next_s.x), idx(next_s.y) tuple 
                # has now been visited, so it can be closed.
                closed[stack_num][idx(next_s.x)][idx(next_s.y)] = 1
                # The next_s came from the current state, and is recorded.
                came_from[stack_num][idx(next_s.x)][idx(next_s.y)] = current
```

代码实现：


次

此算法是非结构化环境中，路径探寻的最佳算法之一。

此类环境的例子有停车场和迷宫等，特点是驾驶规则限制少，速度低，与高速公路以及接到环境殊为不同，这种环境里并不存在明显的参考路径，

也就是哪种车辆90%时间都在上面行驶的路径。因为这种环境中，情况随时在变化。

相较而言，高速公路和街道是相当结构化的环境，所有的驾驶都在预定义规则的限制下进行，而我们的驾驶也可按照某些规则进行，如车辆行驶方向、车道边界、速度限制等。这些规则对车辆行驶加以限制，当我们生成运动轨迹时，也必须满足所有这些规则。

一种在无规则下的最佳算法，如果不能有效利用这些规则，则不适合这种结构化环境。

而对于高速公路这种结构化的环境，我们可以有效利用环境规则，比如以车道线为参考。

# Frenet REminder

用Frenet坐标，可以简化车辆运动模型，

![](./tg107.png)

![](./tg108.png)

考虑三维场景(因场景是随机的，必须考虑时间因素)

![](./tg200.png)

将三维场景分解为两个独立的二维模型

![](./tg201.png)

# 高速公路(固定场景)，轨迹生成方法

我们的目标是生成连续的轨迹，比如下场景，汽车在高速路上匀速行驶，10S后需要驶出高速，即已知当前状态(位置，速度，加速度，终点状态，以及时间)，拟合出最佳的连续曲线。

![](./tg300.png)

拟合的曲线，既要考虑连续性，还要考虑平滑性，否则车辆会给人不舒适感甚至出现危险。

很明显，引起不舒适感的因素是加速度的变化率。

![](./tg301.png)

# Additional Resources on Path Planning

Indoors
Intention-Net: Integrating Planning and Deep Learning for Goal-Directed Autonomous Navigation by S. W. Gao, et. al.
https://arxiv.org/abs/1710.05627

Abstract: How can a delivery robot navigate reliably to a destination in a new office building, with minimal prior information? To tackle this challenge, this paper introduces a two-level hierarchical approach, which integrates model-free deep learning and model-based path planning. At the low level, a neural-network motion controller, called the intention-net, is trained end-to-end to provide robust local navigation. The intention-net maps images from a single monocular camera and "intentions" directly to robot controls. At the high level, a path planner uses a crude map, e.g., a 2-D floor plan, to compute a path from the robot's current location to the goal. The planned path provides intentions to the intention-net. Preliminary experiments suggest that the learned motion controller is robust against perceptual uncertainty and by integrating with a path planner, it generalizes effectively to new environments and goals.


City Navigation

Learning to Navigate in Cities Without a Map by P. Mirowski, et. al.
https://arxiv.org/abs/1804.00168

Abstract: Navigating through unstructured environments is a basic capability of intelligent creatures, and thus is of fundamental interest in the study and development of artificial intelligence. Long-range navigation is a complex cognitive task that relies on developing an internal representation of space, grounded by recognizable landmarks and robust visual processing, that can simultaneously support continuous self-localization ("I am here") and a representation of the goal ("I am going there"). Building upon recent research that applies deep reinforcement learning to maze navigation problems, we present an end-to-end deep reinforcement learning approach that can be applied on a city scale. [...] We present an interactive navigation environment that uses Google StreetView for its photographic content and worldwide coverage, and demonstrate that our learning method allows agents to learn to navigate multiple cities and to traverse to target destinations that may be kilometers away. [...]

Intersections

A Look at Motion Planning for Autonomous Vehicles at an Intersection by S. Krishnan, et. al.

https://arxiv.org/abs/1806.07834

Abstract: Autonomous Vehicles are currently being tested in a variety of scenarios. As we move towards Autonomous Vehicles, how should intersections look? To answer that question, we break down an intersection management into the different conundrums and scenarios involved in the trajectory planning and current approaches to solve them. Then, a brief analysis of current works in autonomous intersection is conducted. With a critical eye, we try to delve into the discrepancies of existing solutions while presenting some critical and important factors that have been addressed. Furthermore, open issues that have to be addressed are also emphasized. We also try to answer the question of how to benchmark intersection management algorithms by providing some factors that impact autonomous navigation at intersection.

Planning in Traffic with Deep Reinforcement Learning

DeepTraffic: Crowdsourced Hyperparameter Tuning of Deep Reinforcement Learning Systems for Multi-Agent Dense Traffic Navigation by L. Fridman, J. Terwilliger and B. Jenik

https://arxiv.org/abs/1801.02805

Abstract: We present a traffic simulation named DeepTraffic where the planning systems for a subset of the vehicles are handled by a neural network as part of a model-free, off-policy reinforcement learning process. The primary goal of DeepTraffic is to make the hands-on study of deep reinforcement learning accessible to thousands of students, educators, and researchers in order to inspire and fuel the exploration and evaluation of deep Q-learning network variants and hyperparameter configurations through large-scale, open competition. This paper investigates the crowd-sourced hyperparameter tuning of the policy network that resulted from the first iteration of the DeepTraffic competition where thousands of participants actively searched through the hyperparameter space.